# NER tagging

# Data download and description

In [ ]:
from urllib.request import urlretrieve
urlretrieve('https://raw.githubusercontent.com/pranabsarkar/Conll_task/master/conll-2003/eng.train','eng.train')
urlretrieve('https://raw.githubusercontent.com/pranabsarkar/Conll_task/master/conll-2003/eng.testa','eng.testa')

istream = open('eng.train')
for idx, line in enumerate(istream):
  print(line.strip())
  if idx >=20:
    break
istream.close()


-DOCSTART- -X- -X- O

EU NNP I-NP I-ORG
rejects VBZ I-VP O
German JJ I-NP I-MISC
call NN I-NP O
to TO I-VP O
boycott VB I-VP O
British JJ I-NP I-MISC
lamb NN I-NP O
. . O O

Peter NNP I-NP I-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP I-NP I-LOC
1996-08-22 CD I-NP O

The DT I-NP O
European NNP I-NP I-ORG
Commission NNP I-NP I-ORG


The CONLL 2003 dataset encodes each token on a single line followed by its annotation:

> (token,tag,chunk,named entity)

The NER tags follow the IOB convention:
* **I** means **Inside** (part of a named entity);
* **B** means **Begin** (starting a new entity);
* **O** means **Outside** (not part of a named entity).

The **I** and **B** Tag are followed by a specifier (for example, PER=person, ORG=Organisation).

# Data preprocessing

In [ ]:
def vocabulary(filename, input_vocab, padding='<pad>', unknown='<unk>'):
    # input_vocab is a boolean flag to tell if we extract input or output vocabulary

    idx2sym = {}
    sym2idx = {}

    cur_idx = 0
    # Add pad and unk tokens to the vocab
    if padding:
      idx2sym[cur_idx] = padding
      sym2idx[padding] = cur_idx
      cur_idx += 1
    if unknown:
      idx2sym[cur_idx] = unknown
      sym2idx[unknown] = cur_idx
      cur_idx += 1

    with open(filename, 'r', encoding='utf-8') as file:
      for line in file:
        line = line.strip()

        if not line or line.startswith('-DOCSTART-'):
          continue  # Skip empty lines and metadata

        parts = line.split()
        if input_vocab:
          token = parts[0]  # The token is the first column
          if token not in sym2idx:
            idx2sym[cur_idx] = token
            sym2idx[token] = cur_idx
            cur_idx += 1
        else:
          tag = parts[-1]  # The NER tag is the last column
          if tag not in sym2idx:
            idx2sym[cur_idx] = tag
            sym2idx[tag] = cur_idx
            cur_idx += 1

    return idx2sym, sym2idx

In [ ]:
def pad_sequence(sequence, pad_size, pad_token):
    # returns a list with pad tokens
    if sequence is None:
      print("None sequence is found and replaced with pad tokens.")
      return [pad_token] * pad_size
    return sequence[:pad_size] + [pad_token] * (pad_size - len(sequence))

def code_sequence(sequence, coding_map, unk_token=None):
    # takes a list of strs and returns a list of ints
    if sequence is None:
      return []
    unk_index = coding_map.get(unk_token, None)
    return [coding_map.get(token, unk_index) for token in sequence]

def decode_sequence(sequence, decoding_map):
    # takes a list of ints and returns a list of strs
    return [decoding_map.get(index, '<unk>') for index in sequence]

In [ ]:
# Testing
# Sample vocab maps
sym2idx = {'EU': 1, 'NNP': 2, 'I-NP': 3, 'I-ORG': 4, '<pad>': 0, '<unk>': 5}
idx2sym = {v: k for k, v in sym2idx.items()}

sequence = ['EU', 'NNP', 'I-NP', 'I-ORG']
pad_size = 6
padded_seq= pad_sequence(sequence, pad_size, pad_token='<pad>')
print(padded_seq)

encoded_seq = code_sequence(padded_seq, sym2idx, unk_token='<unk>')
print(encoded_seq)

# Test decoding
decoded_seq = decode_sequence(encoded_seq, idx2sym)
print(decoded_seq)


['EU', 'NNP', 'I-NP', 'I-ORG', '<pad>', '<pad>']
[1, 2, 3, 4, 0, 0]
['EU', 'NNP', 'I-NP', 'I-ORG', '<pad>', '<pad>']


# Data generator

In [ ]:
def read_conll_tokens(conllfilename):
    # reads a CONLL 2003 file and returns a list of sentences.
    # A sentence is a list of strings (tokens)
    sentences = []
    cur_sent = []

    with open(conllfilename, 'r') as f:
      for line in f:
        line = line.strip()
        if not line:
          if cur_sent:
            sentences.append(cur_sent)
            cur_sent = []
        elif not line.startswith('-DOCSTART-'):
          parts = line.split()
          cur_sent.append(parts[0])   # token

    # Add the last sentence if the file does not end with a newline
    if cur_sent:
      sentences.append(cur_sent)

    return sentences

def read_conll_tags(conllfilename):
    # reads a CONLL 2003 file and returns a list of sentences.
    # A sentence is a list of strings (tokens)
    sentences = []
    cur_sent = []

    with open(conllfilename, 'r') as f:
      for line in f:
        line = line.strip()
        if not line:
          if cur_sent:
            sentences.append(cur_sent)
            cur_sent = []
        elif not line.startswith('-DOCSTART-'):
          parts = line.split()
          cur_sent.append(parts[-1])   # NER tag

    if cur_sent:
      sentences.append(cur_sent)

    return sentences

In [ ]:
# Test on first sentence
sample_tokens = tokens[0]
sample_tags = tags[0]
print(sample_tokens)
print(sample_tags)
print()

# Pad sequences
padded_tokens = pad_sequence(sample_tokens, pad_size=12, pad_token='<pad>')
padded_tags = pad_sequence(sample_tags, pad_size=12, pad_token='<pad>')
print(padded_tokens)
print(padded_tags)
print()

# Encode sequences
encoded_tokens = code_sequence(padded_tokens, input_sym2idx, unk_token='<unk>')
encoded_tags = code_sequence(padded_tags, output_sym2idx, unk_token='<unk>')
print(encoded_tokens)
print(encoded_tags)
print()

# Decode sequences
decoded_tokens = decode_sequence(encoded_tokens, input_idx2sym)
decoded_tags = decode_sequence(encoded_tags, output_idx2sym)
print(decoded_tokens)
print(decoded_tags)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.', '<pad>', '<pad>', '<pad>']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O', '<pad>', '<pad>', '<pad>']

[2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0]
[2, 3, 4, 3, 3, 3, 4, 3, 3, 0, 0, 0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.', '<pad>', '<pad>', '<pad>']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O', '<pad>', '<pad>', '<pad>']




Now we implement the class. You will rely on the helper functions designed above in order to fill in the blanks in the constructor.

In [ ]:
import torch
import torch.nn as nn
from random import shuffle

class DataGenerator:

        def __init__(self,conllfilename, parentgenerator = None, pad_token='<pad>',unk_token='<unk>'):

              if parentgenerator is not None: 
                  self.pad_token = parentgenerator.pad_token
                  self.unk_token = parentgenerator.unk_token
                  self.input_sym2idx = parentgenerator.input_sym2idx
                  self.input_idx2sym = parentgenerator.input_idx2sym
                  self.output_sym2idx = parentgenerator.output_sym2idx
                  self.output_idx2sym = parentgenerator.output_idx2sym
              else:                           # Creates new encodings
                  self.pad_token = pad_token
                  self.unk_token = unk_token
                  # Creates 4 encoding maps from datafile
                  self.input_idx2sym,self.input_sym2idx = vocabulary(conllfilename,input_vocab=True,padding=pad_token,unknown=unk_token)
                  self.output_idx2sym,self.output_sym2idx = vocabulary(conllfilename,input_vocab=False,padding=pad_token,unknown=unk_token)

              # store the conll dataset with sentence structure (a list of lists of strings) in the following fields
              self.Xtokens = read_conll_tokens(conllfilename)
              self.Ytokens = read_conll_tags(conllfilename)

        def generate_batches(self,batch_size):

              # Generator function yielding one batch after another. Batches are lists of lists

              assert(len(self.Xtokens) == len(self.Ytokens))

              N = len(self.Xtokens)
              idxes = list(range(N))

              # Data ordering
              shuffle(idxes)
              idxes.sort(key=lambda idx: len(self.Xtokens[idx]))

              # batch generation
              bstart = 0
              while bstart < N:
                 bend = min(bstart+batch_size,N)
                 batch_idxes = idxes[bstart:bend]
                 batch_len = max(len(self.Xtokens[idx]) for idx in batch_idxes)

                 seqX = [pad_sequence(self.Xtokens[idx],batch_len,self.pad_token) for idx in batch_idxes]
                 seqY = [pad_sequence(self.Ytokens[idx],batch_len,self.pad_token) for idx in batch_idxes]
                 seqX = [code_sequence(seq,self.input_sym2idx,self.unk_token) for seq in seqX]
                 seqY = [code_sequence(seq,self.output_sym2idx) for seq in seqY]
                 #print(seqX, seqY)

                 assert(len(seqX) == len(seqY))
                 yield (seqX,seqY)
                 bstart += batch_size

Implementation of the tagger.

* Implemention of parameter allocation (the embedding layer, the LSTM (or bi-LSTM) layer and the Linear Layer);
* Implemention of the forward method;
* Implemention of the train method.

In [ ]:
import torch.optim as optim

class NERtagger(nn.Module):

      def __init__(self,traingenerator, embedding_size,hidden_size,device='cuda'):
        super(NERtagger, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.allocate_params(traingenerator,device)

      def load(self,filename):
        self.load_state_dict(torch.load(filename))

      def allocate_params(self,datagenerator,device):
        # Get vocabulary and number of output classes
        vocab_size = len(datagenerator.input_sym2idx)  # Size of the input vocabulary
        num_classes = len(datagenerator.output_sym2idx)  # Number of output tags

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, self.embedding_size, padding_idx=datagenerator.input_sym2idx[datagenerator.pad_token]).to(device)

        # Bi-LSTM layer
        self.lstm = nn.LSTM(
          input_size=self.embedding_size,
          hidden_size=self.hidden_size,
          num_layers=1,   # by default
          batch_first=True,
          bidirectional=True  # Enables Bi-LSTM
       ).to(device)

        # Linear layer
        self.linear = nn.Linear(self.hidden_size * 2, num_classes).to(device)  # Bi-LSTM output is 2 * hidden_size

      def forward(self,Xinput):
        # prediction steps
        # Embedding layer
        embeddings = self.embedding(Xinput)  # Shape: (batch_size, seq_len, embedding_size)

        # LSTM layer
        lstm_out, _ = self.lstm(embeddings)  # Shape: (batch_size, seq_len, hidden_size * 2)

        # Linear layer
        logits = self.linear(lstm_out)  # Shape: (batch_size, seq_len, num_classes)

        return logits

      def train_model(self,traingenerator,validgenerator,epochs,batch_size,device='cuda',learning_rate=0.001):

        self.minloss = 10000000 # the min loss found so far on validation data

        self.to(device)
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        pad_index = traingenerator.output_sym2idx[traingenerator.pad_token]
        loss_fnc = nn.CrossEntropyLoss(ignore_index=pad_index)

        for epoch in range(1, epochs + 1):
          print(f"\nEpoch {epoch}/{epochs}")
          self.train()  # Set model to training mode

          batch_losses = []
          batch_accuracies = []

          for seqX, seqY in traingenerator.generate_batches(batch_size):
            X = torch.LongTensor(seqX).to(device)
            Y = torch.LongTensor(seqY).to(device)

            Yhat = self.forward(X)

            # Flatten outputs for loss computation
            batch_size, seq_len = Y.shape
            Yhat = Yhat.view(batch_size * seq_len, -1)
            Y = Y.view(batch_size * seq_len)

            # Loss computation
            loss = loss_fnc(Yhat, Y)
            batch_losses.append(loss.item())

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accuracy computation
            mask = (Y != pad_index)
            Yargmax = torch.argmax(Yhat, dim=1)
            correct = torch.sum((Yargmax == Y) * mask)
            total = torch.sum(mask)
            batch_accuracies.append(float(correct) / float(total))

          # Epoch summary
          train_loss = sum(batch_losses) / len(batch_losses)
          train_accuracy = sum(batch_accuracies) / len(batch_accuracies)
          print(f"[train] Epoch {epoch} mean loss = {train_loss:.4f} | mean accuracy = {train_accuracy:.4f}")

          # Validate model
          valid_loss, valid_accuracy = self.validate(validgenerator, batch_size, device, save_min_model=True)
          print(f"[valid] Epoch {epoch} mean loss = {valid_loss:.4f} | mean accuracy = {valid_accuracy:.4f}")

      def validate(self,datagenerator,batch_size,device='cuda',save_min_model=False):

          batch_accurracies = []
          batch_losses = []

          device = torch.device(device)
          pad_index = datagenerator.output_sym2idx[datagenerator.pad_token]
          loss_fnc = nn.CrossEntropyLoss(ignore_index=pad_index)

          for (seqX,seqY) in datagenerator.generate_batches(batch_size):
                with torch.no_grad():
                  X = torch.LongTensor(seqX).to(device)
                  Y = torch.LongTensor(seqY).to(device)

                  Yhat = self.forward(X)

                  #Flattening and loss computation
                  batch_size,seq_len = Y.shape
                  Yhat = Yhat.view(batch_size*seq_len,-1)
                  Y = Y.view(batch_size*seq_len)
                  loss = loss_fnc(Yhat,Y)
                  batch_losses.append(loss.item())

                  #Accurracy computation
                  mask = (Y != pad_index)
                  Yargmax = torch.argmax(Yhat,dim=1)
                  correct = torch.sum((Yargmax == Y) * mask)
                  total = torch.sum(mask)
                  batch_accurracies.append(float(correct)/float(total))

          L = len(batch_losses)
          valid_loss = sum(batch_losses)/L
          valid_accuracy = sum(batch_accurracies) / L

          if save_min_model and valid_loss < self.minloss:
            self.minloss = valid_loss
            torch.save(self.state_dict(), 'tagger_params.pt')

          return valid_loss, valid_accuracy

# Main program

In [ ]:
# Load the full dataset
full_trainset = DataGenerator('eng.train')

# Create a small sample from the full dataset
sample_size = 15  # Choose a small sample size for testing
small_trainset = DataGenerator('eng.train')

# Limit the Xtokens and Ytokens to the first few examples
small_trainset.Xtokens = full_trainset.Xtokens[:sample_size]
small_trainset.Ytokens = full_trainset.Ytokens[:sample_size]

In [ ]:
# Initialize the model
tagger = NERtagger(small_trainset, embedding_size=64, hidden_size=128, device='cuda')

# Train on the small dataset
tagger.train_model(traingenerator=small_trainset, validgenerator=small_trainset, epochs=4, batch_size=4, device='cuda', learning_rate=0.001)



Epoch 1/4
[train] Epoch 1 mean loss = 2.2258 | mean accuracy = 0.4242
[valid] Epoch 1 mean loss = 2.1146 | mean accuracy = 0.7474

Epoch 2/4
[train] Epoch 2 mean loss = 2.0262 | mean accuracy = 0.7904
[valid] Epoch 2 mean loss = 1.8731 | mean accuracy = 0.8416

Epoch 3/4
[train] Epoch 3 mean loss = 1.7318 | mean accuracy = 0.8416
[valid] Epoch 3 mean loss = 1.4944 | mean accuracy = 0.8434

Epoch 4/4
[train] Epoch 4 mean loss = 1.1980 | mean accuracy = 0.8434
[valid] Epoch 4 mean loss = 0.8577 | mean accuracy = 0.8434


In [ ]:
# Train on the full dataset
trainset = DataGenerator('eng.train')
validset = DataGenerator('eng.testa',parentgenerator = trainset)
tagger   = NERtagger(trainset,embedding_size=64,hidden_size=128,device='cuda')
tagger.train_model(traingenerator=trainset,validgenerator=validset,epochs=10,batch_size=32,device='cuda',learning_rate=0.001)


Epoch 1/10
[train] Epoch 1 mean loss = 0.7072 | mean accuracy = 0.7990
[valid] Epoch 1 mean loss = 0.5355 | mean accuracy = 0.8359

Epoch 2/10
[train] Epoch 2 mean loss = 0.3694 | mean accuracy = 0.8854
[valid] Epoch 2 mean loss = 0.3516 | mean accuracy = 0.8934

Epoch 3/10
[train] Epoch 3 mean loss = 0.2167 | mean accuracy = 0.9320
[valid] Epoch 3 mean loss = 0.2610 | mean accuracy = 0.9199

Epoch 4/10
[train] Epoch 4 mean loss = 0.1244 | mean accuracy = 0.9615
[valid] Epoch 4 mean loss = 0.2249 | mean accuracy = 0.9343

Epoch 5/10
[train] Epoch 5 mean loss = 0.0828 | mean accuracy = 0.9741
[valid] Epoch 5 mean loss = 0.2328 | mean accuracy = 0.9338

Epoch 6/10
[train] Epoch 6 mean loss = 0.0350 | mean accuracy = 0.9892
[valid] Epoch 6 mean loss = 0.2695 | mean accuracy = 0.9304

Epoch 7/10
[train] Epoch 7 mean loss = 0.0156 | mean accuracy = 0.9954
[valid] Epoch 7 mean loss = 0.2839 | mean accuracy = 0.9322

Epoch 8/10
[train] Epoch 8 mean loss = 0.0087 | mean accuracy = 0.9973
[val

# Search for hyperparameters

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00


In [ ]:
#Optuna
import optuna
import time
from copy import deepcopy
import torch

# Subset the dataset for quicker testing
subset_size = 100  # Adjust this to your desired small dataset size
small_trainset = deepcopy(trainset)
small_trainset.Xtokens = trainset.Xtokens[:subset_size]
small_trainset.Ytokens = trainset.Ytokens[:subset_size]

small_validset = deepcopy(validset)
small_validset.Xtokens = validset.Xtokens[:subset_size]
small_validset.Ytokens = validset.Ytokens[:subset_size]

def objective(trial):
    # Suggest hyperparameters using Optuna
    embedding_size = trial.suggest_categorical("embedding_size", [32, 64, 128])
    hidden_size = trial.suggest_categorical("hidden_size", [64, 128, 256])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    print(f"\nTesting configuration: embedding_size={embedding_size}, hidden_size={hidden_size}, "
          f"learning_rate={learning_rate}, batch_size={batch_size}")

    # Create a new model
    tagger = NERtagger(small_trainset, embedding_size, hidden_size, device='cuda')

    # Train the model with the suggested hyperparameters
    tagger.train_model(
        traingenerator=small_trainset,
        validgenerator=small_validset,
        epochs=10,  # Reduced for quicker testing
        batch_size=batch_size,
        device='cuda',
        learning_rate=learning_rate
    )

    # Validate the model and retrieve metrics
    valid_loss, valid_accuracy = tagger.validate(small_validset, batch_size, device='cuda')

    # Return validation loss as the objective to minimize
    return valid_loss

# Measure time for the optimization process
start_time = time.time()

# Create a study and optimize
study = optuna.create_study(direction="minimize")  # Minimize the validation loss
study.optimize(objective, n_trials=20)  # Set number of trials

end_time = time.time()
elapsed_time = end_time - start_time

# Extrapolate based on the size of the dataset
train_size_ratio = len(trainset.Xtokens) / len(small_trainset.Xtokens)
total_time_estimate = elapsed_time * train_size_ratio

# Best result
best_result = study.best_params
best_loss = study.best_value
print("\nBest configuration:")
print(best_result)
print(f"Best validation loss: {best_loss}")
print(f"\nTime taken for small dataset: {elapsed_time:.2f} seconds")
print(f"Estimated time for full dataset: {total_time_estimate / 3600:.2f} hours")


[I 2024-11-20 19:20:40,809] A new study created in memory with name: no-name-68447067-5b2c-4bb3-9555-004b298f84f1
<ipython-input-16-6c99836de519>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)



Testing configuration: embedding_size=32, hidden_size=256, learning_rate=0.0005212933350473318, batch_size=64

Epoch 1/10
[train] Epoch 1 mean loss = 2.2901 | mean accuracy = 0.1721
[valid] Epoch 1 mean loss = 2.2626 | mean accuracy = 0.3625

Epoch 2/10
[train] Epoch 2 mean loss = 2.2114 | mean accuracy = 0.7034
[valid] Epoch 2 mean loss = 2.1904 | mean accuracy = 0.7472

Epoch 3/10
[train] Epoch 3 mean loss = 2.0944 | mean accuracy = 0.8294
[valid] Epoch 3 mean loss = 2.0929 | mean accuracy = 0.7459

Epoch 4/10
[train] Epoch 4 mean loss = 1.8091 | mean accuracy = 0.8189
[valid] Epoch 4 mean loss = 1.8007 | mean accuracy = 0.7600

Epoch 5/10
[train] Epoch 5 mean loss = 1.2415 | mean accuracy = 0.8054
[valid] Epoch 5 mean loss = 1.2991 | mean accuracy = 0.7707

Epoch 6/10
[train] Epoch 6 mean loss = 0.7981 | mean accuracy = 0.8047
[valid] Epoch 6 mean loss = 1.1193 | mean accuracy = 0.7656

Epoch 7/10
[train] Epoch 7 mean loss = 0.7430 | mean accuracy = 0.8056
[valid] Epoch 7 mean loss

[I 2024-11-20 19:20:41,966] Trial 0 finished with value: 0.8417664170265198 and parameters: {'embedding_size': 32, 'hidden_size': 256, 'learning_rate': 0.0005212933350473318, 'batch_size': 64}. Best is trial 0 with value: 0.8417664170265198.


[train] Epoch 9 mean loss = 0.6438 | mean accuracy = 0.8060
[valid] Epoch 9 mean loss = 1.0591 | mean accuracy = 0.7506

Epoch 10/10
[train] Epoch 10 mean loss = 0.6071 | mean accuracy = 0.8180
[valid] Epoch 10 mean loss = 1.0580 | mean accuracy = 0.7504

Testing configuration: embedding_size=64, hidden_size=64, learning_rate=0.0023836895341926394, batch_size=16

Epoch 1/10
[train] Epoch 1 mean loss = 2.2154 | mean accuracy = 0.3679
[valid] Epoch 1 mean loss = 2.0755 | mean accuracy = 0.6899

Epoch 2/10
[train] Epoch 2 mean loss = 1.1800 | mean accuracy = 0.8399
[valid] Epoch 2 mean loss = 1.0087 | mean accuracy = 0.7852

Epoch 3/10
[train] Epoch 3 mean loss = 0.6489 | mean accuracy = 0.8212
[valid] Epoch 3 mean loss = 0.9118 | mean accuracy = 0.7946

Epoch 4/10
[train] Epoch 4 mean loss = 0.5128 | mean accuracy = 0.8599
[valid] Epoch 4 mean loss = 0.8890 | mean accuracy = 0.7955

Epoch 5/10
[train] Epoch 5 mean loss = 0.3894 | mean accuracy = 0.8888
[valid] Epoch 5 mean loss = 0.8900 

[I 2024-11-20 19:20:42,940] Trial 1 finished with value: 1.1463214755058289 and parameters: {'embedding_size': 64, 'hidden_size': 64, 'learning_rate': 0.0023836895341926394, 'batch_size': 16}. Best is trial 0 with value: 0.8417664170265198.


...

[valid] Epoch 10 mean loss = 1.1155 | mean accuracy = 0.7907

Testing configuration: embedding_size=64, hidden_size=64, learning_rate=0.0008725106985317709, batch_size=32

Epoch 1/10
[train] Epoch 1 mean loss = 2.2408 | mean accuracy = 0.2203
[valid] Epoch 1 mean loss = 2.2176 | mean accuracy = 0.3454

Epoch 2/10
[train] Epoch 2 mean loss = 1.9905 | mean accuracy = 0.7824
[valid] Epoch 2 mean loss = 1.8340 | mean accuracy = 0.7800

Epoch 3/10
[train] Epoch 3 mean loss = 1.1241 | mean accuracy = 0.8156
[valid] Epoch 3 mean loss = 1.1543 | mean accuracy = 0.7798

Epoch 4/10
[train] Epoch 4 mean loss = 0.7228 | mean accuracy = 0.8157
[valid] Epoch 4 mean loss = 1.0644 | mean accuracy = 0.8007

Epoch 5/10
[train] Epoch 5 mean loss = 0.6386 | mean accuracy = 0.8304
[valid] Epoch 5 mean loss = 1.0264 | mean accuracy = 0.8003

Epoch 6/10
[train] Epoch 6 mean loss = 0.5774 | mean accuracy = 0.8312
[valid] Epoch 6 mean loss = 1.0021 | mean accuracy = 0.7971

Epoch 7/10
[train] Epoch 7 mean loss

[I 2024-11-20 19:21:01,484] Trial 18 finished with value: 0.8394912332296371 and parameters: {'embedding_size': 64, 'hidden_size': 64, 'learning_rate': 0.0008725106985317709, 'batch_size': 32}. Best is trial 16 with value: 0.7878151386976242.


[train] Epoch 9 mean loss = 0.4058 | mean accuracy = 0.8797
[valid] Epoch 9 mean loss = 0.9881 | mean accuracy = 0.7700

Epoch 10/10
[train] Epoch 10 mean loss = 0.3469 | mean accuracy = 0.8934
[valid] Epoch 10 mean loss = 1.0057 | mean accuracy = 0.7635

Testing configuration: embedding_size=64, hidden_size=256, learning_rate=0.0002878359641906546, batch_size=32

Epoch 1/10
[train] Epoch 1 mean loss = 2.2489 | mean accuracy = 0.4788
[valid] Epoch 1 mean loss = 2.2234 | mean accuracy = 0.6789

Epoch 2/10
[train] Epoch 2 mean loss = 2.0642 | mean accuracy = 0.7972
[valid] Epoch 2 mean loss = 1.9284 | mean accuracy = 0.7855

Epoch 3/10
[train] Epoch 3 mean loss = 1.2904 | mean accuracy = 0.8248
[valid] Epoch 3 mean loss = 1.2705 | mean accuracy = 0.7847

Epoch 4/10
[train] Epoch 4 mean loss = 0.8934 | mean accuracy = 0.8257
[valid] Epoch 4 mean loss = 1.1347 | mean accuracy = 0.8046

Epoch 5/10
[train] Epoch 5 mean loss = 0.7790 | mean accuracy = 0.8364
[valid] Epoch 5 mean loss = 1.0763

[I 2024-11-20 19:21:02,922] Trial 19 finished with value: 0.7641179263591766 and parameters: {'embedding_size': 64, 'hidden_size': 256, 'learning_rate': 0.0002878359641906546, 'batch_size': 32}. Best is trial 19 with value: 0.7641179263591766.


[valid] Epoch 10 mean loss = 0.9740 | mean accuracy = 0.7951

Best configuration:
{'embedding_size': 64, 'hidden_size': 256, 'learning_rate': 0.0002878359641906546, 'batch_size': 32}
Best validation loss: 0.7641179263591766

Time taken for small dataset: 22.12 seconds
Estimated time for full dataset: 0.86 hours


In [ ]:
import optuna
import time
import torch

# Load the full dataset
trainset = DataGenerator('eng.train')  # Full training dataset
validset = DataGenerator('eng.testa', parentgenerator=trainset)  # Validation dataset

def objective(trial):
    # Suggest hyperparameters using Optuna
    embedding_size = trial.suggest_categorical("embedding_size", [32, 64, 128])
    hidden_size = trial.suggest_categorical("hidden_size", [64, 128, 256])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    print(f"\nTesting configuration: embedding_size={embedding_size}, hidden_size={hidden_size}, "
          f"learning_rate={learning_rate}, batch_size={batch_size}")

    # Create a new model
    tagger = NERtagger(trainset, embedding_size, hidden_size, device='cuda')

    # Train the model with the suggested hyperparameters
    tagger.train_model(
        traingenerator=trainset,
        validgenerator=validset,
        epochs=10,  # Number of epochs
        batch_size=batch_size,
        device='cuda',
        learning_rate=learning_rate
    )

    # Validate the model and retrieve metrics
    valid_loss, valid_accuracy = tagger.validate(validset, batch_size, device='cuda')

    # Return validation loss as the objective to minimize
    return valid_loss

# Measure time for the optimization process
start_time = time.time()

# Create a study and optimize
study = optuna.create_study(direction="minimize")  # Minimize the validation loss
study.optimize(objective, n_trials=20)  # Number of trials

end_time = time.time()
elapsed_time = end_time - start_time

# Best result
best_result = study.best_params
best_loss = study.best_value
print("\nBest configuration:")
print(best_result)
print(f"Best validation loss: {best_loss}")
print(f"\nTotal time taken: {elapsed_time:.2f} seconds")

[I 2024-11-20 19:21:16,237] A new study created in memory with name: no-name-74c36813-6f9f-4337-8b3f-d622490fb0d4
<ipython-input-17-c298c45a410c>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)



Testing configuration: embedding_size=128, hidden_size=128, learning_rate=0.0060505333502602655, batch_size=16

Epoch 1/10
[train] Epoch 1 mean loss = 0.2839 | mean accuracy = 0.9166
[valid] Epoch 1 mean loss = 0.2285 | mean accuracy = 0.9303

Epoch 2/10
[train] Epoch 2 mean loss = 0.1050 | mean accuracy = 0.9678
[valid] Epoch 2 mean loss = 0.2113 | mean accuracy = 0.9464

Epoch 3/10
[train] Epoch 3 mean loss = 0.1044 | mean accuracy = 0.9715
[valid] Epoch 3 mean loss = 0.3355 | mean accuracy = 0.9073

Epoch 4/10
[train] Epoch 4 mean loss = 0.0753 | mean accuracy = 0.9792
[valid] Epoch 4 mean loss = 0.3110 | mean accuracy = 0.9222

Epoch 5/10
[train] Epoch 5 mean loss = 0.0579 | mean accuracy = 0.9846
[valid] Epoch 5 mean loss = 0.3496 | mean accuracy = 0.9238

Epoch 6/10
[train] Epoch 6 mean loss = 0.0464 | mean accuracy = 0.9878
[valid] Epoch 6 mean loss = 0.3535 | mean accuracy = 0.9308

Epoch 7/10
[train] Epoch 7 mean loss = 0.0422 | mean accuracy = 0.9892
[valid] Epoch 7 mean los

[I 2024-11-20 19:27:59,652] Trial 0 finished with value: 0.42292219349274446 and parameters: {'embedding_size': 128, 'hidden_size': 128, 'learning_rate': 0.0060505333502602655, 'batch_size': 16}. Best is trial 0 with value: 0.42292219349274446.


[valid] Epoch 10 mean loss = 0.4216 | mean accuracy = 0.9398

Testing configuration: embedding_size=128, hidden_size=128, learning_rate=0.00516785550973882, batch_size=32

Epoch 1/10
[train] Epoch 1 mean loss = 0.3524 | mean accuracy = 0.8949
[valid] Epoch 1 mean loss = 0.2529 | mean accuracy = 0.9222

Epoch 2/10
[train] Epoch 2 mean loss = 0.1044 | mean accuracy = 0.9671
[valid] Epoch 2 mean loss = 0.2062 | mean accuracy = 0.9430

Epoch 3/10
[train] Epoch 3 mean loss = 0.0225 | mean accuracy = 0.9935
[valid] Epoch 3 mean loss = 0.2409 | mean accuracy = 0.9484

Epoch 4/10
[train] Epoch 4 mean loss = 0.0259 | mean accuracy = 0.9921
[valid] Epoch 4 mean loss = 0.2815 | mean accuracy = 0.9443

Epoch 5/10
[train] Epoch 5 mean loss = 0.0749 | mean accuracy = 0.9797
[valid] Epoch 5 mean loss = 0.3596 | mean accuracy = 0.9232

Epoch 6/10
[train] Epoch 6 mean loss = 0.0530 | mean accuracy = 0.9851
[valid] Epoch 6 mean loss = 0.3535 | mean accuracy = 0.9243

Epoch 7/10
[train] Epoch 7 mean loss

[I 2024-11-20 19:33:06,467] Trial 1 finished with value: 0.5394770292674794 and parameters: {'embedding_size': 128, 'hidden_size': 128, 'learning_rate': 0.00516785550973882, 'batch_size': 32}. Best is trial 0 with value: 0.42292219349274446.


...

[valid] Epoch 10 mean loss = 0.2882 | mean accuracy = 0.9481

Testing configuration: embedding_size=64, hidden_size=256, learning_rate=0.00020082829074530813, batch_size=64

Epoch 1/10
[train] Epoch 1 mean loss = 1.1038 | mean accuracy = 0.7334
[valid] Epoch 1 mean loss = 0.9120 | mean accuracy = 0.7764

Epoch 2/10
[train] Epoch 2 mean loss = 0.6856 | mean accuracy = 0.8087
[valid] Epoch 2 mean loss = 0.6244 | mean accuracy = 0.8172

Epoch 3/10
[train] Epoch 3 mean loss = 0.4899 | mean accuracy = 0.8545
[valid] Epoch 3 mean loss = 0.5036 | mean accuracy = 0.8461

Epoch 4/10
[train] Epoch 4 mean loss = 0.3790 | mean accuracy = 0.8859
[valid] Epoch 4 mean loss = 0.4234 | mean accuracy = 0.8693

Epoch 5/10
[train] Epoch 5 mean loss = 0.2862 | mean accuracy = 0.9113
[valid] Epoch 5 mean loss = 0.3625 | mean accuracy = 0.8760

Epoch 6/10
[train] Epoch 6 mean loss = 0.1877 | mean accuracy = 0.9415
[valid] Epoch 6 mean loss = 0.3014 | mean accuracy = 0.8950

Epoch 7/10
[train] Epoch 7 mean lo

[I 2024-11-20 20:58:53,082] Trial 18 finished with value: 0.3206683176697469 and parameters: {'embedding_size': 64, 'hidden_size': 256, 'learning_rate': 0.00020082829074530813, 'batch_size': 64}. Best is trial 13 with value: 0.2262512398701088.


[valid] Epoch 10 mean loss = 0.3125 | mean accuracy = 0.9090

Testing configuration: embedding_size=32, hidden_size=64, learning_rate=0.0003426719736850518, batch_size=32

Epoch 1/10
[train] Epoch 1 mean loss = 0.9998 | mean accuracy = 0.7547
[valid] Epoch 1 mean loss = 0.8542 | mean accuracy = 0.7741

Epoch 2/10
[train] Epoch 2 mean loss = 0.7207 | mean accuracy = 0.7846
[valid] Epoch 2 mean loss = 0.7050 | mean accuracy = 0.7970

Epoch 3/10
[train] Epoch 3 mean loss = 0.5636 | mean accuracy = 0.8268
[valid] Epoch 3 mean loss = 0.6065 | mean accuracy = 0.8273

Epoch 4/10
[train] Epoch 4 mean loss = 0.4325 | mean accuracy = 0.8667
[valid] Epoch 4 mean loss = 0.5376 | mean accuracy = 0.8520

Epoch 5/10
[train] Epoch 5 mean loss = 0.3163 | mean accuracy = 0.9010
[valid] Epoch 5 mean loss = 0.4845 | mean accuracy = 0.8729

Epoch 6/10
[train] Epoch 6 mean loss = 0.2285 | mean accuracy = 0.9274
[valid] Epoch 6 mean loss = 0.4476 | mean accuracy = 0.8947

Epoch 7/10
[train] Epoch 7 mean loss

[I 2024-11-20 21:03:17,825] Trial 19 finished with value: 0.4403449323247461 and parameters: {'embedding_size': 32, 'hidden_size': 64, 'learning_rate': 0.0003426719736850518, 'batch_size': 32}. Best is trial 13 with value: 0.2262512398701088.


[valid] Epoch 10 mean loss = 0.4331 | mean accuracy = 0.9226

Best configuration:
{'embedding_size': 64, 'hidden_size': 128, 'learning_rate': 0.0001847078780156269, 'batch_size': 64}
Best validation loss: 0.2262512398701088

Total time taken: 6121.59 seconds


# Improvements

To improve the NER Tagger we firsly need to modify `vocabulary` function by adding there POS tag and Characters mapping, to add `read_conll_pos` function for POS tags, to add new function for caharcters extraction, a new `datagenerator` with pos_sym2idx, pos_idx2sym, char_sym2idx, char_idx2sym.

In [ ]:
# NEW VOCABULARY with pos_tag and char mapping
def vocabulary(filename, vocab_type, padding='<pad>', unknown='<unk>'):
    # vocab_type tells the type of vocabulary to create: 'input', 'output', 'pos', 'char'
    # the optional flags indicate that a padding and an unknown token have to be added to the vocabulary
    # if their value is not None

    idx2sym = {}
    sym2idx = {}

    cur_idx = 0
    # Add pad and unk tokens to the vocab if applicable
    if padding:
      idx2sym[cur_idx] = padding
      sym2idx[padding] = cur_idx
      cur_idx += 1
    if unknown:
      idx2sym[cur_idx] = unknown
      sym2idx[unknown] = cur_idx
      cur_idx += 1

    with open(filename, 'r', encoding='utf-8') as f:
      for line in f:
        line = line.strip()

        if not line or line.startswith('-DOCSTART-'):
          continue  # Skip empty lines and metadata

        parts = line.split()
        if vocab_type == 'input':
          token = parts[0]  # Token
          if token not in sym2idx:
            idx2sym[cur_idx] = token
            sym2idx[token] = cur_idx
            cur_idx += 1

        elif vocab_type == 'output':
          tag = parts[-1]  # NER tag
          if tag not in sym2idx:
            idx2sym[cur_idx] = tag
            sym2idx[tag] = cur_idx
            cur_idx += 1

        elif vocab_type == 'pos':
          pos_tag = parts[1]  # POS tag
          if pos_tag not in sym2idx:
            idx2sym[cur_idx] = pos_tag
            sym2idx[pos_tag] = cur_idx
            cur_idx += 1

        elif vocab_type == 'char':
          token = parts[0]  # Token
          for char in token:  # Extract all characters from the token
            if char not in sym2idx:
              idx2sym[cur_idx] = char
              sym2idx[char] = cur_idx
              cur_idx += 1

    return idx2sym, sym2idx

In [ ]:
# ADD read_conll function for POS tags
def read_conll_pos(conllfilename):
    """
    Reads a CONLL 2003 file and returns a list of sentences.
    A sentence is a list of strings (POS tags)
    """
    #TODO
    sentences = []
    cur_sent = []

    with open(conllfilename, 'r') as f:
      for line in f:
        line = line.strip()
        if not line:
          if cur_sent:
            sentences.append(cur_sent)
            cur_sent = []
        elif not line.startswith('-DOCSTART-'):
        #else:
          parts = line.split()
          cur_sent.append(parts[1])   # POS tag

    if cur_sent:
      sentences.append(cur_sent)

    return sentences

In [ ]:
# NEW function to extract characters
def extract_char_sequences(tokens):
    char_sequences = []
    for sentence in tokens:
      char_sequences.append([[char for char in token] for token in sentence])
    return char_sequences

In [ ]:
# Test on first sentence Pad sequences, Encode sequences and Decode sequences
sample_tokens = tokens[0]
sample_tags = tags[0]
sample_pos = pos_tags[0]
sample_chars = char_sequences[0]
print(sample_tokens)
print(sample_tags)
print(sample_pos)
print(sample_chars)
print()

# Pad sequences
padded_tokens = pad_sequence(sample_tokens, pad_size=12, pad_token='<pad>')
padded_tags = pad_sequence(sample_tags, pad_size=12, pad_token='<pad>')
padded_pos = pad_sequence(sample_pos, pad_size=12, pad_token='<pad>')
padded_chars = [pad_sequence(char_seq, pad_size=15, pad_token='<pad>') for char_seq in sample_chars]
print(padded_tokens)
print(padded_tags)
print(padded_pos)
print(padded_chars)
print()

# Encode sequences
encoded_tokens = code_sequence(padded_tokens, input_sym2idx, unk_token='<unk>')
encoded_tags = code_sequence(padded_tags, output_sym2idx, unk_token='<unk>')
encoded_pos = code_sequence(padded_pos, pos_sym2idx, unk_token='<unk>')
encoded_chars = [code_sequence(char_seq, char_sym2idx, unk_token='<unk>') for char_seq in padded_chars]
print(encoded_tokens)
print(encoded_tags)
print(encoded_pos)
print(encoded_chars)
print()

# Decode sequences
decoded_tokens = decode_sequence(encoded_tokens, input_idx2sym)
decoded_tags = decode_sequence(encoded_tags, output_idx2sym)
decoded_pos = decode_sequence(encoded_pos, pos_idx2sym)
decoded_chars = [decode_sequence(char_seq, char_idx2sym) for char_seq in encoded_chars]
print(decoded_tokens)
print(decoded_tags)
print(decoded_pos)
print(decoded_chars)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']
['NNP', 'VBZ', 'JJ', 'NN', 'TO', 'VB', 'JJ', 'NN', '.']
[['E', 'U'], ['r', 'e', 'j', 'e', 'c', 't', 's'], ['G', 'e', 'r', 'm', 'a', 'n'], ['c', 'a', 'l', 'l'], ['t', 'o'], ['b', 'o', 'y', 'c', 'o', 't', 't'], ['B', 'r', 'i', 't', 'i', 's', 'h'], ['l', 'a', 'm', 'b'], ['.']]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.', '<pad>', '<pad>', '<pad>']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O', '<pad>', '<pad>', '<pad>']
['NNP', 'VBZ', 'JJ', 'NN', 'TO', 'VB', 'JJ', 'NN', '.', '<pad>', '<pad>', '<pad>']
[['E', 'U', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['r', 'e', 'j', 'e', 'c', 't', 's', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['G', 'e', 'r', 'm', 'a', 'n', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [ ]:
# NEW datagenerator with pos_sym2idx, pos_idx2sym, char_sym2idx, char_idx2sym
import torch
import torch.nn as nn
from random import shuffle

class DataGenerator:

        def __init__(self,conllfilename, parentgenerator = None, pad_token='<pad>',unk_token='<unk>'):

              if parentgenerator is not None:
                  self.pad_token = parentgenerator.pad_token
                  self.unk_token = parentgenerator.unk_token
                  self.input_sym2idx = parentgenerator.input_sym2idx
                  self.input_idx2sym = parentgenerator.input_idx2sym
                  self.output_sym2idx = parentgenerator.output_sym2idx
                  self.output_idx2sym = parentgenerator.output_idx2sym
                  self.pos_sym2idx = parentgenerator.pos_sym2idx
                  self.pos_idx2sym = parentgenerator.pos_idx2sym
                  self.char_sym2idx = parentgenerator.char_sym2idx
                  self.char_idx2sym = parentgenerator.char_idx2sym
              else:                           # Creates new encodings
                  self.pad_token = pad_token
                  self.unk_token = unk_token
                  #TODO : Create 8 encoding maps from datafile
                  self.input_idx2sym, self.input_sym2idx = vocabulary(conllfilename, vocab_type='input', padding=pad_token, unknown=unk_token)
                  self.output_idx2sym, self.output_sym2idx = vocabulary(conllfilename, vocab_type='output', padding=pad_token, unknown=unk_token)
                  self.pos_idx2sym, self.pos_sym2idx = vocabulary(conllfilename, vocab_type='pos', padding=pad_token, unknown=unk_token)
                  self.char_idx2sym, self.char_sym2idx = vocabulary(conllfilename, vocab_type='char', padding=pad_token, unknown=unk_token)

              #TODO : store the conll dataset with sentence structure (a list of lists of strings) in the following fields
              self.Xtokens = read_conll_tokens(conllfilename)
              self.Ytokens = read_conll_tags(conllfilename)
              self.Xpos = read_conll_pos(conllfilename)
              self.Xchars = extract_char_sequences(self.Xtokens)

        def generate_batches(self,batch_size):

              # Generator function yielding one batch after another. Batches are lists of lists

              assert(len(self.Xtokens) == len(self.Ytokens) == len(self.Xpos) == len(self.Xchars))

              N = len(self.Xtokens)
              idxes = list(range(N))

              # Data ordering
              shuffle(idxes)
              idxes.sort(key=lambda idx: len(self.Xtokens[idx]))

              # batch generation
              bstart = 0
              while bstart < N:
                 bend        = min(bstart+batch_size,N)
                 batch_idxes = idxes[bstart:bend]
                 batch_len   = max(len(self.Xtokens[idx]) for idx in batch_idxes)

                 # Pad sequences (tokens, NER tags, POS tags)
                 seqX = [pad_sequence(self.Xtokens[idx],batch_len,self.pad_token) for idx in batch_idxes]
                 seqY = [pad_sequence(self.Ytokens[idx],batch_len,self.pad_token) for idx in batch_idxes]
                 seqPOS = [pad_sequence(self.Xpos[idx], batch_len, self.pad_token) for idx in batch_idxes]

                 # Encode sequences (tokens, NER tags, POS tags)
                 seqX = [code_sequence(seq,self.input_sym2idx,self.unk_token) for seq in seqX]
                 seqY = [code_sequence(seq,self.output_sym2idx) for seq in seqY]
                 seqPOS = [code_sequence(seq, self.pos_sym2idx, self.unk_token) for seq in seqPOS]
                 #print(seqX, seqY, seqPOS)

                 # Character sequences
                 char_batch = []
                 for i in batch_idxes:
                   char_sequences = self.Xchars[i]  # list of character sequences for each token in a sentence
                   padded_char_sequences = [pad_sequence(char_seq, pad_size=15, pad_token=self.pad_token) for char_seq in char_sequences]
                   encoded_char_sequences = [code_sequence(char_seq, self.char_sym2idx, self.unk_token) for char_seq in padded_char_sequences]
                   char_batch.append(encoded_char_sequences)

                 # Pad character sequences for each word in the sentence
                 max_sent_len = batch_len
                 max_word_len = 15
                 # Ensure all sentences in the batch have the same length for character sequences
                 seqChar = [pad_sequence(char_seq, max_sent_len, [self.char_sym2idx[self.pad_token]] * max_word_len) for char_seq in char_batch]

                 assert(len(seqX) == len(seqY) == len(seqPOS) == len(seqChar))
                 yield (seqX,seqY,seqPOS, seqChar)
                 bstart += batch_size

# Improved NER Tagger

In [ ]:
# NEW NER Tagger
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class NERtagger(nn.Module):

    def __init__(self, traingenerator, embedding_size, hidden_size, pos_embedding_size, char_embedding_size, cnn_filters, device='cuda'):
        super(NERtagger, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.pos_embedding_size = pos_embedding_size
        self.char_embedding_size = char_embedding_size
        self.cnn_filters = cnn_filters
        self.allocate_params(traingenerator, device)

    def allocate_params(self, datagenerator, device):
        # Vocabulary and output sizes
        vocab_size = len(datagenerator.input_sym2idx)
        num_classes = len(datagenerator.output_sym2idx)
        pos_vocab_size = len(datagenerator.pos_sym2idx)
        char_vocab_size = len(datagenerator.char_sym2idx)

        # Token embeddings
        self.embedding = nn.Embedding(vocab_size, self.embedding_size, padding_idx=datagenerator.input_sym2idx[datagenerator.pad_token]).to(device)

        # POS embeddings
        self.pos_embedding = nn.Embedding(pos_vocab_size, self.pos_embedding_size, padding_idx=datagenerator.pos_sym2idx[datagenerator.pad_token]).to(device)

        # Character embeddings and CNN (1D convolution)
        self.char_embedding = nn.Embedding(char_vocab_size, self.char_embedding_size, padding_idx=datagenerator.char_sym2idx[datagenerator.pad_token]).to(device)
        self.char_cnn = nn.Conv1d(self.char_embedding_size, self.cnn_filters, kernel_size=3, padding=1).to(device)

        # Bi-LSTM
        self.lstm = nn.LSTM(
            input_size=self.embedding_size + self.pos_embedding_size + self.cnn_filters,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        ).to(device)

        # Attention layer
        self.attention = nn.Linear(self.hidden_size * 2, 1, bias=False).to(device)

        # Output layer
        self.linear = nn.Linear(self.hidden_size * 2, num_classes).to(device)

    def compute_attention(self, lstm_out):
        # lstm_out: (batch_size, seq_len, hidden_size * 2)
        attention_scores = self.attention(lstm_out)  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len, 1)
        context = torch.sum(attention_weights * lstm_out, dim=1)  # Shape: (batch_size, hidden_size * 2)
        return context

    def forward(self, Xinput, Xpos, Xchar):
        # Embedding layers
        token_embeddings = self.embedding(Xinput)  # Shape: (batch_size, seq_len, embedding_size)
        pos_embeddings = self.pos_embedding(Xpos)  # Shape: (batch_size, seq_len, pos_embedding_size)

        # Character embeddings + CNN
        batch_size, seq_len, word_len = Xchar.shape
        Xchar_flat = Xchar.view(-1, word_len)  # Shape: (batch_size * seq_len, word_len)
        char_embeddings = self.char_embedding(Xchar_flat).permute(0, 2, 1)  # Shape: (batch_size * seq_len, char_embedding_size, word_len)
        char_features = torch.relu(self.char_cnn(char_embeddings))  # Shape: (batch_size * seq_len, cnn_filters, word_len)
        # max pooling operation
        char_features = torch.max(char_features, dim=2).values  # Shape: (batch_size * seq_len, cnn_filters)
        char_features = char_features.view(batch_size, seq_len, -1)  # Shape: (batch_size, seq_len, cnn_filters)

        # Concatenate features
        embeddings = torch.cat([token_embeddings, pos_embeddings, char_features], dim=2)  # Shape: (batch_size, seq_len, embedding_size + pos_embedding_size + cnn_filters)

        # Bi-LSTM
        lstm_out, _ = self.lstm(embeddings)  # Shape: (batch_size, seq_len, hidden_size * 2)

        # Attention
        context = self.compute_attention(lstm_out)  # Shape: (batch_size, hidden_size * 2)

        # Output Layer
        logits = self.linear(lstm_out)  # Shape: (batch_size, seq_len, num_classes)
        return logits

    def train_model(self, traingenerator, validgenerator, epochs, batch_size, device='cuda', learning_rate=0.001):
        self.minloss = float('inf')
        self.to(device)
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        pad_index = traingenerator.output_sym2idx[traingenerator.pad_token]
        loss_fnc = nn.CrossEntropyLoss(ignore_index=pad_index)

        for epoch in range(1, epochs + 1):
            print(f"\nEpoch {epoch}/{epochs}")
            self.train()
            batch_losses, batch_accuracies = [], []

            for seqX, seqY, seqPOS, seqChar in traingenerator.generate_batches(batch_size):
                X = torch.LongTensor(seqX).to(device)
                Y = torch.LongTensor(seqY).to(device)
                POS = torch.LongTensor(seqPOS).to(device)
                CHAR = torch.LongTensor(seqChar).to(device)

                Yhat = self.forward(X, POS, CHAR)

                # Flatten for loss computation
                batch_size, seq_len = Y.shape
                Yhat = Yhat.view(batch_size * seq_len, -1)
                Y = Y.view(batch_size * seq_len)

                # Compute loss
                loss = loss_fnc(Yhat, Y)
                batch_losses.append(loss.item())

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Compute accuracy
                mask = (Y != pad_index)
                Yargmax = torch.argmax(Yhat, dim=1)
                correct = torch.sum((Yargmax == Y) * mask)
                total = torch.sum(mask)
                batch_accuracies.append(float(correct) / float(total))

            # Epoch summary
            train_loss = sum(batch_losses) / len(batch_losses)
            train_accuracy = sum(batch_accuracies) / len(batch_accuracies)
            print(f"[train] Epoch {epoch} mean loss = {train_loss:.4f} | mean accuracy = {train_accuracy:.4f}")

            valid_loss, valid_accuracy = self.validate(validgenerator, batch_size, device, save_min_model=True)
            print(f"[valid] Epoch {epoch} mean loss = {valid_loss:.4f} | mean accuracy = {valid_accuracy:.4f}")

    def validate(self, datagenerator, batch_size, device='cuda', save_min_model=False):
        batch_losses, batch_accuracies = [], []
        pad_index = datagenerator.output_sym2idx[datagenerator.pad_token]
        loss_fnc = nn.CrossEntropyLoss(ignore_index=pad_index)

        for seqX, seqY, seqPOS, seqChar in datagenerator.generate_batches(batch_size):
            with torch.no_grad():
                X = torch.LongTensor(seqX).to(device)
                Y = torch.LongTensor(seqY).to(device)
                POS = torch.LongTensor(seqPOS).to(device)
                CHAR = torch.LongTensor(seqChar).to(device)

                Yhat = self.forward(X, POS, CHAR)

                # Flatten for loss computation
                batch_size, seq_len = Y.shape
                Yhat = Yhat.view(batch_size * seq_len, -1)
                Y = Y.view(batch_size * seq_len)

                # Compute loss
                loss = loss_fnc(Yhat, Y)
                batch_losses.append(loss.item())

                # Compute accuracy
                mask = (Y != pad_index)
                Yargmax = torch.argmax(Yhat, dim=1)
                correct = torch.sum((Yargmax == Y) * mask)
                total = torch.sum(mask)
                batch_accuracies.append(float(correct) / float(total))

        # Validation summary
        valid_loss = sum(batch_losses) / len(batch_losses)
        valid_accuracy = sum(batch_accuracies) / len(batch_accuracies)

        if save_min_model and valid_loss < self.minloss:
            self.minloss = valid_loss
            torch.save(self.state_dict(), 'tagger_params.pt')

        return valid_loss, valid_accuracy

In [ ]:
# Train on the full dataset
trainset_i = DataGenerator('eng.train')
validset_i = DataGenerator('eng.testa',parentgenerator = trainset_i)
tagger_i = NERtagger(trainset_i,embedding_size=64,hidden_size=128,pos_embedding_size=32,char_embedding_size=25,cnn_filters=30,device='cuda')
tagger_i.train_model(traingenerator=trainset_i,validgenerator=validset_i,epochs=13,batch_size=64,device='cuda',learning_rate=0.0001847)


Epoch 1/13
[train] Epoch 1 mean loss = 0.8721 | mean accuracy = 0.7670
[valid] Epoch 1 mean loss = 0.5775 | mean accuracy = 0.8145

Epoch 2/13
[train] Epoch 2 mean loss = 0.4182 | mean accuracy = 0.8625
[valid] Epoch 2 mean loss = 0.4084 | mean accuracy = 0.8557

Epoch 3/13
[train] Epoch 3 mean loss = 0.2785 | mean accuracy = 0.9116
[valid] Epoch 3 mean loss = 0.2738 | mean accuracy = 0.9194

Epoch 4/13
[train] Epoch 4 mean loss = 0.2009 | mean accuracy = 0.9389
[valid] Epoch 4 mean loss = 0.2088 | mean accuracy = 0.9377

Epoch 5/13
[train] Epoch 5 mean loss = 0.1463 | mean accuracy = 0.9549
[valid] Epoch 5 mean loss = 0.1897 | mean accuracy = 0.9424

Epoch 6/13
[train] Epoch 6 mean loss = 0.1001 | mean accuracy = 0.9693
[valid] Epoch 6 mean loss = 0.1543 | mean accuracy = 0.9547

Epoch 7/13
[train] Epoch 7 mean loss = 0.0696 | mean accuracy = 0.9792
[valid] Epoch 7 mean loss = 0.1386 | mean accuracy = 0.9577

Epoch 8/13
[train] Epoch 8 mean loss = 0.0479 | mean accuracy = 0.9860
[val

# Conclusion

**Hyperparameters search**
We used the hyperparameter optimization framework Optuna and we searched for the best configuration of hyperparameters by minimizing the validation loss.

The best configuration found is: {'embedding_size': 64, 'hidden_size': 128, 'learning_rate': 0.0001847078780156269, 'batch_size': 64} with the best validation loss: 0.23.

The training results are as follows:\
The train loss: 0.0535\
Tthe train accuracy: 98.50%\
The validation results are as follows:\
The valid loss: 0.2172\
The valid accuracy: 93.58%

The model maintains a reasonably good balance between memorizing training data and generalizing to unseen validation data. The difference between the validation accuracy (93.58%) and the training accuracy (98.50%) is 5%, indicating minor overfitting but still reasonable generalizability.

**NER Tagger improvements**

1) **Attention layer.** We added an attention layer to enhance the model's focus on key parts of the input sequence. This layer takes the Bi-LSTM output and produces a single attention score for each token. The softmax function is used to normalize these scores into `attention_weights`. The `context` vector then is computed as a weighted sum of the Bi-LSTM outputs, providing a richer representation in the `forward()` method.

2) **Part-of-speech (POS) tags embeddings.** We incorporated part-of-speech tag embeddings as additional inputs. First a separate embedding layer is defined for the POS tags, then in the `forward()` method POS tags are embedded using the POS embedding layer, and finally POS embeddings are concatenated with the token embeddings and character-level features.

3) **Character-level embeddings with Convolutional module for Unknown words.** To handle unknown words, we used a convolutional neural network (CNN) over character embeddings. First an embedding for each character in the vocabulary is created, then it is passed through the convolutional neural network CNN that applies filters to extract meaningful subword caracter-level features. As a result the model can generate an embedding for an unknown word by combining the features learned from its characters.

**Results**
With the improved model the best validation loss was achieved at the Epoch 9: 0.1279.
The train accuracy achieved is 99.06%.
The valid accuracy achieved is 96.34%.
After improvements the valid accuracy increased by 2.76% from 93.58% to 96.34%. These results demonstrate that the implementation of the attention mechanism, of the POS tag embeddings, and of the convolutional word embedding module improved the NER Tagger model's ability to generalize and accurately recognize named entities, making it more efficient.


